In [1]:
import pulp

# Supón que ya tienes el archivo "All.txt" con las líneas que mostraste.
# Cada línea es:  x_n  y_n  z_n  w_n
# Podrías cargarlo así (o crear la lista a mano si lo prefieres):

data = []
with open('All.txt', 'r') as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        x_str, y_str, z_str, w_str = line.split()
        x_val = float(x_str)
        y_val = float(y_str)
        z_val = float(z_str)
        w_val = float(w_str)
        data.append((x_val, y_val, z_val, w_val))

# Verifica cuántos puntos leíste
N = len(data)
print("Cantidad de puntos:", N)

# ----------------------------
# Definición del polinomio F(x,y,z) = sum_{i=0..1} sum_{j=0..2} sum_{k=0..3} c_{i,j,k} x^i y^j z^k
# Tenemos i en [0,1], j en [0..2], k en [0..3], por lo que hay 2 * 3 * 4 = 24 coeficientes c_{ijk}.
# Queremos minimizar el error máximo:  min E  s.t.  |F(x_n,y_n,z_n) - w_n| <= E   para todo n.

# 1) Creamos un problema de optimización
problem = pulp.LpProblem("AjusteMinimax", pulp.LpMinimize)

# 2) Definimos las variables c_{i,j,k} y la variable E (error máximo)
c = {}
for i in range(2):  # 0..1
    for j in range(3):  # 0..2
        for k in range(4):  # 0..3
            c[(i,j,k)] = pulp.LpVariable(f"c_{i}_{j}_{k}",
                                         lowBound=None,  # sin límite inferior
                                         upBound=None,   # sin límite superior
                                         cat=pulp.LpContinuous)

# E >= 0, representa la cota suprema del error absoluto
E = pulp.LpVariable("E", lowBound=0, cat=pulp.LpContinuous)

# 3) Función objetivo: minimizar E
problem += E, "Minimize_maximum_error"

# 4) Restricciones: para cada punto (x_n, y_n, z_n, w_n):
#    -E <= sum_{i,j,k} c_{i,j,k} * x_n^i * y_n^j * z_n^k - w_n <= E
for (x_n, y_n, z_n, w_n) in data:
    # Construimos la expresión polinomial F(x_n, y_n, z_n)
    poly_value = pulp.lpSum([
        c[(i,j,k)] * (x_n**i) * (y_n**j) * (z_n**k)
        for i in range(2)
        for j in range(3)
        for k in range(4)
    ])
    
    # Restricción:  poly_value - w_n <= E
    problem += (poly_value - w_n <= E)
    # Restricción: -(poly_value - w_n) <= E  =>  w_n - poly_value <= E
    problem += (-(poly_value - w_n) <= E)

# 5) Resolver con algún solver LP (ej. el default CBC)
problem.solve(pulp.PULP_CBC_CMD(msg=0))

# 6) Extraer resultados
c_values = {key: pulp.value(var) for key, var in c.items()}
E_value = pulp.value(E)

print("===== Resultados =====")
print("Coeficientes c_{i,j,k}:")
for key in sorted(c_values.keys()):
    print(f"c{key} = {c_values[key]:.6f}")

print(f"\nError máximo minimizado E = {E_value:.6f}")


Cantidad de puntos: 50
===== Resultados =====
Coeficientes c_{i,j,k}:
c(0, 0, 0) = -0.049827
c(0, 0, 1) = 0.825094
c(0, 0, 2) = 0.310347
c(0, 0, 3) = -0.155729
c(0, 1, 0) = 0.705212
c(0, 1, 1) = -0.995438
c(0, 1, 2) = -1.549765
c(0, 1, 3) = 0.461470
c(0, 2, 0) = -0.753364
c(0, 2, 1) = 0.746985
c(0, 2, 2) = 1.450566
c(0, 2, 3) = -0.384050
c(1, 0, 0) = 0.544841
c(1, 0, 1) = -1.128027
c(1, 0, 2) = 0.044762
c(1, 0, 3) = 0.065149
c(1, 1, 0) = -0.652821
c(1, 1, 1) = 4.246204
c(1, 1, 2) = -0.451415
c(1, 1, 3) = -0.168936
c(1, 2, 0) = 0.292430
c(1, 2, 1) = -3.733850
c(1, 2, 2) = 0.406260
c(1, 2, 3) = 0.126002

Error máximo minimizado E = 0.195920
